# JAX 소개

2024년 여름 대학생을 위한 인공지능 튜토리얼 - 과학기계학습 맛보기

2024년 7월 16일 실습

## 들어가기

### 동기

일반적으로 사용하는 파이썬<sub>(정확히는 CPython 기준)</sub>은 성능을 비롯한 여러 가지 이유로, 과학 계산에는 그리 적합하지 않습니다.

In [1]:
a_py = [-1.0, -0.5, 0.0, 0.5, 1.0]
b_py = [1.0, 2.0, 3.0, 4.0, 5.0]

# 벡터(리스트) 더하기...?
a_py + b_py

[-1.0, -0.5, 0.0, 0.5, 1.0, 1.0, 2.0, 3.0, 4.0, 5.0]

In [2]:
# 벡터 원소별로 더하기
c = []
for x, y in zip(a_py, b_py):
    c.append(x + y)
# 또는
# c = [x + y for x, y in zip(a_py, b_py)]
c

[0.0, 1.5, 3.0, 4.5, 6.0]

대신 파이썬은 확장성이 매우 좋아서, 파이썬을 사용하면서도 과학 계산에 적합<sub>(주요 연산 부분은 다른 언어를 이용하거나 하는 등의 방법으로 성능을 높임)</sub>하도록 라이브러리가 많이 개발되어 있고, [NumPy](https://numpy.org/)는 파이썬의 가장 대표적인 행렬 연산 라이브러리입니다.

In [4]:
import numpy as np

a_np = np.array([-1.0, -0.5, 0.0, 0.5, 1.0])
b_np = np.array([1.0, 2.0, 3.0, 4.0, 5.0])

a_np + b_np

array([0. , 1.5, 3. , 4.5, 6. ])

NumPy를 잘 이용하여 벡터화(병렬화)된 연산을 하면 CPU의 여러 코어를 잘 쓸 수 있지만, NumPy는 CPU가 아닌 GPU를 이용한 연산은 (적어도 아직까지는) 지원하지 않습니다.

### JAX란?

> JAX is a Python library for accelerator-oriented array computation and program transformation, designed for high-performance numerical computing and large-scale machine learning.
>
> ― [JAX documentation](https://jax.readthedocs.io/en/latest/)

**기능**

- 가속기(GPU, TPU)를 지원하는 행렬 연산 라이브러리(NumPy 대체) `jax.numpy`
- Just-in-time(JIT) 컴파일 `jax.jit()`
- 자동 미분 계산 `jax.grad()`, `jax.jacfwd()`, `jax.jacrev()` 등
- 자동 벡터화(병렬화) `jax.vmap()`, `jax.pmap()`

**참고**

- JAX가 딥 러닝만을 위한 라이브러리는 아닙니다.
  NumPy로 사용하던 기존의 과학(수치) 계산도 JAX로 구현하여 GPU 사용이나 JIT 사용의 이점을 얻을 수 있습니다.
- JAX 자체가 TensorFlow나 PyTorch의 대체제는 아닙니다.
  JAX를 기반으로 작동하는 기계 학습 라이브러리로 [Flax (Google)](https://github.com/google/flax), ~~[Haiku (DeepMind)](https://github.com/deepmind/dm-haiku)~~(deprecated), [Equinox](https://github.com/patrick-kidger/equinox) 등이 있습니다.
- PyTorch에서도 [PyTorch/XLA](https://github.com/pytorch/xla), [functorch](https://github.com/pytorch/functorch) 등 여러가지를 시도하고 있습니다.

### 준비

이번 튜토리얼에서는 Google Colab을 사용합니다.

- Colab에는 기본적으로 JAX가 설치되어 있고, GPU를 사용하기 위해서 Runtime -> Change runtime type에서 GPU를 선택하면 됩니다.
- Colab 무료 사용의 경우 GPU를 배정받지 못하는 경우가 있는데 그럴 경우 JAX는 CPU 모드로 잘 동작하고, 이번 실습에서는 벤치마크 성능이 떨어지는 것 외에 큰 지장은 없습니다.

로컬에 설치하려면 [Installing JAX — JAX documentation](https://jax.readthedocs.io/en/latest/installation.html)을 참고하세요.

**모듈 불러오기**

보통 다음과 같이 `jax`와 함께 `jax.numpy`를 `jnp`로 불러 와서 사용하는 것이 일반적입니다.

In [5]:
import jax
import jax.numpy as jnp
import numpy as np

# 기타 필요한 모듈과 설정
from functools import partial
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
plt.rcParams.update({"figure.constrained_layout.use": True})

현재 JAX에서 인식하여 기본값으로 사용하는 디바이스를 확인합니다.
GPU를 사용 중이라면 `cuda`라고 떠야 합니다.

In [6]:
jax.devices()

[CudaDevice(id=0)]

## 주요 기능

### 행렬 연산 라이브러리: NumPy의 대체제 `jax.numpy`

> Key Concepts:
>
> -   JAX provides a NumPy-inspired interface for convenience.
> -   Through duck-typing, JAX arrays can **often be used as drop-in replacements** of NumPy arrays.
> -   Unlike NumPy arrays, JAX arrays are **always immutable**.
>
> ― [How to Think in JAX](https://jax.readthedocs.io/en/latest/notebooks/thinking_in_jax.html)

대부분의 NumPy API가 JAX에도 똑같이 구현되어있고, 단순 바꿔치기(drop-in replacement)만으로도 잘 동작합니다.

In [7]:
def my_function(x):
    return x + x**2 + x**3

In [8]:
# Python number
my_function(5)

155

In [9]:
[my_function(x) for x in [5, 6, 7, 8, 9]]

[155, 258, 399, 584, 819]

In [10]:
# NumPy array
my_function(np.array([5, 6, 7, 8, 9]))

array([155, 258, 399, 584, 819])

In [11]:
# JAX array
my_function(jnp.array([5, 6, 7, 8, 9]))

Array([155, 258, 399, 584, 819], dtype=int32)

#### 차이점: 데이터형

JAX의 기본 부동소수점 데이터형은 32비트(`float32`)입니다.

64비트 부동소수점 연산을 하려면 처음 시작할 때 JAX 모듈을 불러온 직후

```python
jax.config.update("jax_enable_x64", True)
```

라고 추가로 지정해 주어야 합니다.

In [12]:
# NumPy
np.array([1.0, 2.0]).dtype

dtype('float64')

In [13]:
# JAX
jnp.array([1.0, 2.0]).dtype

dtype('float32')

#### 차이점: JAX array는 변경 불가능(immutable)

JAX에서는 행렬 중 일부 값을 바꾸려면 할당(예: `x[3] = 5`)이 아닌 다른 방법을 써야 하고, 항상 원래의 행렬은 그대로 두고 복사본을 만들어 줍니다.

행렬이 값이 바뀔 수 있으면 JAX가 프로그램을 최적화할 때 더 분석하기가 어렵기 때문에 행렬을 변경하지 않는 것과 (뒤에서 얘기할) 부수 효과가 없는 함수를 만드는 것이 중요합니다.
실제로 JAX가 최적화할 때에는 (가능한 경우) 행렬을 실제로 복사하지 않고 동작하도록 바꿔 주므로, 필요 없는 복사 때문에 생기는 성능 저하는 걱정할 필요가 없습니다.

In [14]:
# NumPy
x = np.arange(4)  # [0, 1, 2, 3]
x[2] = 999
x

array([  0,   1, 999,   3])

In [ ]:
# # JAX: Error
# # 참고: 전체 선택 후 Windows는 Ctrl+/, macOS는 Command+/로
# # 선택한 줄 전체 주석 처리/제거 가능
# x = jnp.arange(4)
# x[2] = 999
# # => TypeError: (...) does not support item assignment. JAX arrays are immutable.

In [15]:
# JAX
x = jnp.arange(4)
new_x = x.at[2].set(999)
print(f"{x = }")
print(f"{new_x = }")

x = Array([0, 1, 2, 3], dtype=int32)
new_x = Array([  0,   1, 999,   3], dtype=int32)


#### 차이점: 가속기(GPU) 자동 사용 + 비동기 디스패치(asynchronous dispatch)

PyTorch의 경우, GPU에서 연산을 하려면 데이터를 (메인) 메모리에서 GPU 메모리로 명시적으로 넘겨준 다음 계산을 해야 하지만, JAX는 자동으로 알아서 처리해 줍니다.

또한 JAX의 연산은 GPU의 계산이 끝나고 결과가 나올 때까지 기다리지 않고(“비동기”), ‘미래에 결과가 들어가게 될 변수’(future)를 먼저 반환하면서 바로 Python에게 제어를 넘겨 줍니다.

전부 자동으로 알아서 처리해 주기 때문에 보통은 거의 의식할 필요는 없지만, 벤치마크시에는 주의가 필요합니다.
벤치마크를 위해서는 연산 후 `.block_until_ready()` 메소드를 써서 계산이 끝날 때까지 강제로 기다리게 할 수 있습니다.

In [16]:
rng = np.random.default_rng(78)

x_np = rng.normal(size=(4000, 4000)).astype(np.float32)
x_jnp = jnp.array(x_np)

print(f"{type(x_np) = }")
print(f"{type(x_jnp) = }")

type(x_np) = <class 'numpy.ndarray'>
type(x_jnp) = <class 'jaxlib.xla_extension.ArrayImpl'>


In [17]:
%time _ = x_np @ x_np

CPU times: user 3.06 s, sys: 70.9 ms, total: 3.14 s
Wall time: 1.63 s


In [18]:
# 주의: 이 셀은 여러 번 실행하면 결과가 많이 변할 수도 있음
%time _ = x_jnp @ x_jnp

CPU times: user 1.9 s, sys: 98.1 ms, total: 2 s
Wall time: 2.95 s


In [19]:
%time _ = x_jnp @ x_jnp

CPU times: user 672 µs, sys: 97 µs, total: 769 µs
Wall time: 459 µs


In [ ]:
%time _ = (x_jnp @ x_jnp).block_until_ready()

In [ ]:
%time y = x_jnp @ x_jnp  # 계산이 다 끝나지 않아도 다음 줄로 이동
%time print(y[0, 0])  # y의 값을 알려면 계산이 다 끝나기를 ‘반드시’ 기다려야 함
%time print(y[0, 0])  # 이미 계산은 끝나 있고 한 번 더 y 값을 읽기만 하면 됨

In [ ]:
import time

%time y = x_jnp @ x_jnp
time.sleep(1)  # Python으로(CPU로) 다른 일 하기(여기서는 편의상 1초 잠들기로 대체)
%time print(y[0, 0])  # 이미 계산이 끝나 있으므로 기다리는 시간 없음
%time print(y[0, 0])

참고: NumPy 함수/array와 JAX array/함수를 섞어 사용하는 경우, 에러가 나지는 않고

- NumPy array를 JAX 함수에 넣으면: 자동으로 JAX array로 바꾼 후 함수를 실행
- JAX array를 NumPy 함수에 넣으면: 자동으로 NumPy array로 바꾼 후 함수를 실행

이렇게 동작합니다.
보통 섞어서 사용할 일은 흔하지 않고, 사용하지 않는 것을 추천합니다.
(SciPy같이 외부 라이브러리가 JAX를 지원하지 않는 경우엔 물론 NumPy로 바꿨다가 다시 되돌려야 합니다.
그런 경우 말고도 둘을 꼭 같이 섞어 써야 하는 경우가 일부 존재는 하지만, 이번 실습에서는 다루지 않습니다.)

꼭 필요한 경우

- `np.array(some_jnp_array)` (JAX -> NumPy 항상 복사)
- `np.asarray(some_jnp_array)` (JAX가 CPU를 쓰던 경우에는 복사하지 않음)
- `jnp.array(some_np_array)` (NumPy -> JAX도 동일)
- `jnp.asarray(some_np_array)`

로 변환할 수 있습니다.

In [ ]:
type(jnp.exp(np.ones(3)))

In [ ]:
type(np.exp(jnp.ones(3)))

#### 예제: 선형 최소자승법(선형 대수)

데이터 $\{(x_i, y_i)\}_{i = 1}^N$가 주어졌을 때, 데이터를 표현하는 모델을 가정하고, 데이터를 가장 잘 표현하는(맞추는) 파라미터를 찾고 싶습니다.

직선 **모델**

$$
y = f(x; b, w) = b + wx
$$

을 사용하고, 평균 제곱 오차를 최소화하고 싶으면, 잘 알려진 선형 최소자승법이 됩니다.
즉, **손실 함수**(loss function, 또는 비용 함수 cost function)

$$
L(b, w) \mathop{:=} \frac1N \sum_{i = 1}^N (y_i - f(x_i; b, w))^2 = \frac1N \sum_{i = 1}^N (y_i - (b + wx_i))^2
$$

이 최소가 되게 하는 $b$, $w$를 찾으려고 합니다.

이를 행렬로 표현하면

$$
L(b, w) = \frac1N \lVert y - X\theta \rVert^2 \\
\text{where} \quad
y = \begin{pmatrix}
    y_1 \\
    y_2 \\
    \vdots \\
    y_N \\
\end{pmatrix}, \quad
X = \begin{pmatrix}
    1 & x_1 \\
    1 & x_2 \\
    \vdots & \vdots \\
    1 & x_N \\
\end{pmatrix}, \quad
\theta = \begin{pmatrix} b \\ w \end{pmatrix}
$$

가 되어, $X^\mathsf{T}X\theta = X^\mathsf{T}y$를 풀어 $L$을 최소화하는 $b$, $w$를 찾을 수 있습니다.

테스트를 위해 $b$, $w$를 미리 지정하고 노이즈가 포함된 샘플 데이터를 만듭니다.

In [ ]:
def generate_linear_samples(key, intercept, slope, noise_scale, n_samples):
    """노이즈가 포함된 직선의 샘플 데이터를 생성한다."""
    # 난수 만들기는 추후에 다시 설명합니다.
    key_x, key_noise = jax.random.split(key)

    x = jax.random.uniform(key_x, shape=(n_samples,))
    y = slope * x + intercept
    y_with_noise = (
        y + noise_scale * jax.random.normal(key_noise, shape=x.shape)
    )

    return x, y_with_noise

In [ ]:
b_true = 4.5
w_true = 1.3
noise_scale = 0.02
n_samples = 15

key = jax.random.key(78)
x_train, y_train = generate_linear_samples(
    key,
    intercept=b_true,
    slope=w_true,
    noise_scale=noise_scale,
    n_samples=n_samples,
)
print(f"{x_train = }")
print(f"{y_train = }")

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x_train, y_train, c="C0", label="Data")
ax.axline([0, b_true], slope=w_true, c="C1", label="True line")
ax.set(
    xlabel=R"$x$",
    ylabel=R"$y$",
    title=Rf"$y = {b_true} + {w_true}x + \epsilon$, $\epsilon \sim \mathcal{{N}}(0, {noise_scale}^2)$",
)
ax.legend()
pass

참고: NumPy와 JAX의 어레이는 행 우선(row-major) 순서이므로 1D 어레이는 보통은 $1 \times N$ 행렬처럼 동작합니다.
단, 행렬곱의 오른쪽 원소가 1D 어레이인 경우에는 예외적으로 열 벡터($N \times 1$ 행렬)처럼 생각하고 곱한 후, 다시 원래의 열 모양처럼 바꿔 줍니다.

In [ ]:
A = jnp.array([[1, 2], [3, 4]])
one_column_matrix = jnp.array([[10], [11]])

A @ one_column_matrix

In [ ]:
row_vector = jnp.array([10, 11])
row_vector

In [ ]:
jnp.atleast_2d(row_vector)

In [ ]:
(A @ jnp.atleast_2d(row_vector).T).T

In [ ]:
A @ row_vector

선형 최소자승법 행렬 연산을 이용하여 모델의 파라미터 $b$, $w$를 찾습니다.

- [jax.numpy.ones — JAX documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.ones.html)
- [jax.numpy.column_stack — JAX documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.column_stack.html)
- [jax.Array — JAX documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.Array.html) (아래쪽 Attributes에 `.T` 설명 있음)
- [jax.numpy.matmul — JAX documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.matmul.html) (`@`)
- [jax.numpy.linalg.solve — JAX documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.linalg.solve.html)


In [ ]:
X = jnp.column_stack((jnp.ones(x_train.shape), x_train))
# 참고: 1D array는 행렬곱의 오른쪽에 있을 때에는 열벡터처럼 쓰임
b_pred, w_pred = jnp.linalg.solve(X.T @ X, X.T @ y_train)
b_pred, w_pred

In [ ]:
# # 참고: jnp.linalg.lstsq()를 이용하여
# (b_pred, w_pred), *_ = jnp.linalg.lstsq(X, y_train)
# b_pred, w_pred
# # 라고 할 수도 있음

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x_train, y_train, c="C0", label="Data")
ax.axline([0, b_true], slope=w_true, c="C1", label="True line")
ax.axline([0, b_pred], slope=w_pred, c="C2", ls="--", label="Fitted line")
ax.set(
    xlabel=R"$x$",
    ylabel=R"$y$",
    title=Rf"Result of linear least squares",
)
ax.legend()
pass

### 실행시 컴파일 (Just-in-time compilation, JIT) `jax.jit()`

JIT을 이용해, 함수를 더 빠르게 동작하도록 최적화할 수 있습니다.

여러 연산을 합치거나, 순서를 바꾸거나 하는 등 다양한 최적화를 자동으로 해 주고, CPU든 GPU든 현재 디바이스에 적합하게 컴파일해 줍니다.

In [ ]:
x_np = np.linspace(0, 1, num=50_000_000, dtype=np.float32)
x_jnp = jnp.array(x_np)

In [ ]:
# NumPy
def f_np(x):
    return np.sin(10 * np.exp(x)) * np.cos(x**2 + 1)

%timeit _ = f_np(x_np)

In [ ]:
# JAX
def f_jnp(x):
    return jnp.sin(10 * jnp.exp(x)) * jnp.cos(x**2 + 1)

# 아직은 JIT 컴파일이 아님: GPU 연산으로 빨라짐
%timeit _ = f_jnp(x_jnp).block_until_ready()

`jax.jit()`의 **인자로 함수를 넣으**면, 새로운 **함수를 반환**해 줍니다.

In [ ]:
f_jitted = jax.jit(f_jnp)

위의 `f_jitted`를 `f_jnp` 대신 똑같이 함수로 사용할 수 있습니다.

정확히는 아직 JIT 컴파일을 한 것은 아니고, 처음 정말로 인자를 넣어서 `f_jitted` 함수를 부를 때에 컴파일을 합니다.
그러므로 처음 실행할 때에는 컴파일 시간이 걸리기 때문에 시간이 오래 걸릴 수 있습니다.

In [ ]:
%time _ = f_jitted(x_jnp).block_until_ready()

같은 종류(뒤에서 더 자세히 설명)의 인자로 다시 실행하면, 컴파일을 다시 하지 않으므로 빠르게 실행됩니다.

In [ ]:
%timeit _ = f_jitted(x_jnp).block_until_ready()

실제로는 꼭 변수에 넣지 않고, `jax.jit()`을 다시 부르더라도, (`f.jnp`를 다시 선언하거나 하지만 않았다면) 알아서 캐시해 줍니다.

In [ ]:
%timeit _ = jax.jit(f_jnp)(x_jnp).block_until_ready()
%timeit _ = jax.jit(f_jnp)(x_jnp).block_until_ready()

데코레이터(decorator) 문법도 사용 가능합니다.

```python
@jax.jit
def f(x):
    ...
```

이라고 하면

```python
def f(x):
    ...
f = jax.jit(f)
```
라고 한 것과 (거의) 동일합니다.
(대신 이미 이름을 덮어썼기 때문에 JIT 컴파일 하기 전의 원래 함수에는 접근할 수 없습니다.)

In [ ]:
@jax.jit
def f(x):
    return jnp.sin(10 * jnp.exp(x)) * jnp.cos(x**2 + 1)

f(x_jnp).block_until_ready()  # 컴파일하기 위해 한 번 실행
%timeit _ = f(x_jnp).block_until_ready()

#### 제한 사항

모든 함수를 컴파일할 수 있지는 않고, 제한 사항이 있습니다.

- 부수 효과(side effect)가 없는 함수, 즉 순수 함수(pure function)여야 함
- 함수는 변수 행렬의 데이터형과 크기에**만** 맞춰 컴파일함
  - 행렬의 크기나 데이터형이 달라지면 새로 컴파일함
  - 계산 과정에 나오는 모든 행렬의 크기가 항상 일정해야 함
  - 변수의 **값**을 이용해 인덱싱을 하거나 **파이썬 제어문**(분기, 반복)을 사용할 수 없음

**순수 함수여야 함**

순수 함수가 아니면(부수 효과가 있으면) 컴파일되지 않거나, 의도하지 않은 결과가 나올 수 있습니다.

In [ ]:
# 보통 함수
def double(x):
    print(f"double is called with {x = }")
    return 2 * x

y = double(2)  # 함수 실행: 함수가 실행될 때 print문을 실행함

print("--------")

print(f"{y = }")

print("--------")

y = double(3)  # 함수 실행

print("--------")

print(f"{y = }")

In [ ]:
# JIT 컴파일된 함수
@jax.jit
def double_jitted(x):
    print(f"double_jitted is called with {x = }")  # 출력도 부수 효과임
    return 2 * x

y = double_jitted(2)
# => 첫 실행시 컴파일: 컴파일할 때 함수를 한 번 실행함.
# 하지만 출력되는 x 변수를 보면 호출자가 넘겨준 2가 아님

print("--------")

print(f"{y = }")

print("--------")

y = double_jitted(3)
# => 다시 컴파일하지 않음: 원래 함수를 실행하는 것이 아니라 출력이 되지 않음

print("--------")

print(f"{y = }")

전역(global) 변수나 비지역(nonlocal) 변수같이 함수 밖에서 정의된 값을 사용(closure)할 수는 있으나, 해당 변수 값이 바뀌지 않아야(부수 효과가 없어야) 의도한 대로 동작합니다. (정확히는 JIT 컴파일하는 시점에 값을 복사해서 “고정해” 둡니다.)

In [ ]:
# 보통 함수
def plus_a(x):
    # a는 함수의 인자에도 없고, 함수 안에서 정의한 적도
    # 없으므로(local 변수가 아님) 함수 밖에서 a를 찾아 와서 계산함.
    # (함수 밖에도 없으면 당연히 에러)
    print(f"plus_a is called with {a = }, {x = }")
    return x + a

a = 10
print(f"현재 {a = }")
print(f"{plus_a(5) = }")
# => 함수를 부를 때 그 때의 a의 값을 읽어 옴

print("--------")

a = 20
print(f"현재 {a = }")
print(f"{plus_a(5) = }")
# => 함수를 부를 때 그 때의 (바뀌어 있는) a의 값을 읽어 옴

In [ ]:
# JIT 컴파일된 함수
@jax.jit
def plus_a_jitted(x):
    print(f"plus_a_jitted is called with {a = }, {x = }")
    return x + a

a = 10
print(f"현재 {a = }")
print(f"{plus_a_jitted(5) = }")
# => 함수를 처음 부를 때(JIT 컴파일할 때) a의 값을 읽어서
# JIT 컴파일한 함수 안에 “그대로 고정함”

print("--------")

a = 20
print(f"현재 {a = }")
print(f"{plus_a_jitted(5) = }")
# => 함수를 부를 때의 a의 값과 상관없이 이전에 JIT 컴파일할 때 고정한 값을 사용함

**함수는 변수 행렬의 데이터형과 크기에‘만’ 맞춰 컴파일함**

컴파일 시에 행렬의 데이터형과 크기만 아는 상태에서 함수가 하는 일을 알 수 있어야 합니다.

In [ ]:
@jax.jit
def half(x):
    print(f"Compiling with {x = }")  # 컴파일할 때만 출력됨
    return x / 2

# 원소가 int32이고 크키가 (3,)인 입력 행렬
y = half(jnp.array([1, 2, 3]))
print(f"{y = }")

print("--------")

# 데이터형과 크기가 같으므로 다시 컴파일하지 않음
y = half(jnp.array([4, 5, 6]))
print(f"{y = }")

print("--------")

# 새로운 데이터형 float32
y = half(jnp.array([1.0, 2.0, 3.0]))
print(f"{y = }")

print("--------")

# 새로운 크기 (4,)
y = half(jnp.array([1, 2, 3, 4]))
print(f"{y = }")

**변수 값을 이용해 인덱싱하거나 파이썬 제어문(분기, 반복)을 사용할 수 없음**

함수의 중간 변수나 최종 반환 변수의 데이터형과 크기가 컴파일 시에 확정되어야 하고, 값에 의해 크기가 변해서는 안됩니다.

In [ ]:
def sum_of_positive_numbers(x):
    positive_numbers = x[x > 0]
    return positive_numbers.sum()

# JIT 컴파일 하지 않으면 문제 없음
sum_of_positive_numbers(jnp.array([1, -1, 2, 0, -3]))

In [ ]:
# jax.jit(sum_of_positive_numbers)(jnp.array([1, -1, 2, 0, -3]))
# # => NonConcreteBooleanIndexError: Array boolean indices must be concrete

인덱싱을 사용하는 기능 중 일부는 `jnp.where`을 이용하면, JIT 컴파일할 수 있습니다.

- [jax.numpy.where — JAX documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.where.html)

In [ ]:
@jax.jit
def sum_of_positive_numbers(x):
    positive_or_zero = jnp.where(x > 0, x, 0)
    return positive_or_zero.sum()

sum_of_positive_numbers(jnp.array([1, -1, 2, 0, -3]))

변수의 값을 이용해 분기(`if` 문)를 할 수 없습니다.

In [ ]:
@jax.jit
def square(x, negate):
    if negate:
        return -x**2
    else:
        return x**2

# square(1, True)  # => TracerBoolConversionError

제어에 쓰려고 하는 변수를 **static**으로 처리하면 컴파일 가능합니다.
단 static 인자는 값이 달라져도 새로 컴파일하므로 값이 계속 바뀌는 변수를 static으로 처리하면 안됩니다(컴파일하는 시간이 더 걸림).

참고: Python의 `functools.partial()`을 사용하면 `jax.jit()`에 추가 인자를 넘겨주어야 할 때에도 데코레이터 문법을 사용할 수 있습니다.

```python
@partial(jax.jit, static_argnums=(1,))
def f(x, y):
    ...
```

이라고 하면

```python
def f(x, y):
    ...
f = jax.jit(f, static_argnums(1,))
```
라고 한 것과 (거의) 동일합니다.

In [ ]:
# Static으로 처리하면 가능
# 해당 값이 다르면 별개 함수인 것처럼 컴파일함
@partial(jax.jit, static_argnums=(1,))  # 또는 static_argnames=("negate",)
def square(x, negate):
    print(f"Compiling with {negate = }.")
    if negate:
        return -x**2
    else:
        return x**2

print(square(jnp.array(1.0), True))

print("--------")

print(square(jnp.array(2.0), True))

print("--------")

print(square(jnp.array(2.0), False))

또는 `jax.lax.cond()` 함수를 이용할 수도 있습니다.

- [jax.lax.cond — JAX documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.lax.cond.html)

In [ ]:
@jax.jit
def square(x, negate):
    print(f"Compiling with {negate = }.")
    return jax.lax.cond(negate, lambda a: -a**2, lambda a: a**2, x)

print(square(jnp.array(1.0), True))

print("--------")

print(square(jnp.array(2.0), True))

print("--------")

print(square(jnp.array(2.0), False))

반복문을 제어하는 변수도 static 처리를 하면 사용 가능합니다.
단 이 경우에는 반복문을 전부 풀기(unroll) 때문에 주의해야 합니다.
대신 반복문 중 일부 경우 `jax.lax.fori_loop()`, `jax.lax.while_loop()`, `jax.lax.scan()`를 사용할 수 있습니다.

- [jax.lax.fori_loop — JAX documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.lax.fori_loop.html)
- [jax.lax.while_loop — JAX documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.lax.while_loop.html)
- [jax.lax.scan — JAX documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.lax.scan.html)

In [ ]:
@jax.jit
def sum_init(x, num):
    """x의 앞쪽 num개의 원소의 합."""
    s = 0
    for i in range(num):
        s = s + x[i]
    return s

# sum_init(jnp.arange(100), 5)  # => TracerIntegerConversionError

In [ ]:
# 아래는
#     s = 0
#     s = s + x[0]
#     s = s + x[1]
#     ...
#     s = s + x[num - 1]
# 이라고 전부 일일이 적은 것과 마찬가지인 함수를 만듦.
# 그리고 위의 예와 같이, `num`의 값이 바뀌면 새로 컴파일함.
@partial(jax.jit, static_argnums=(1,))
def sum_init(x, num):
    s = 0
    for i in range(num):
        s = s + x[i]
    return s

sum_init(jnp.arange(100), 5)

In [ ]:
# 참고
jax.make_jaxpr(sum_init, static_argnums=(1,))(jnp.arange(100), 5)

In [ ]:
@jax.jit
def sum_init(x, num):
    return jax.lax.fori_loop(0, num, lambda i, s: s + x[i], 0)

sum_init(jnp.arange(100), 5)

#### 주의 사항

**컴파일러 최적화에 따라 결과가 달라질 수 있음**

JIT 컴파일러는 식을 보고 수학적으로 동일한 다른(더 빠르거나 더 정확한) 식으로 바꾸거나, 연산 순서를 바꾸는 등의 최적화를 진행합니다.
이 과정에서 부동소수점 연산의 오차로 인해 미세하게 값이 바뀌거나, 오버플로우같은 예외적인 상황의 처리가 달라질 수 있습니다.

In [ ]:
def f(x):
    return jnp.log(jnp.sqrt(x))

x = 3.14

print(f"Normal: {f(x)}")
print(f"Jitted: {jax.jit(f)(x)}")

In [ ]:
# 현재는 다음과 같이 변환시켜 줌(보장된 것은 아님)
print(f"0.5log(x): {0.5 * jnp.log(x)}")

In [ ]:
def f(x):
    return jnp.log(jnp.exp(x))

x = jnp.array([1.0, 10.0, 100.0, 1000.0])

print(f"Normal: {f(x)}")
print(f"Jitted: {jax.jit(f)(x)}")

#### 예제: 경사 하강법

이전 예제인 비용 함수

$$
L(b, w) \mathop{:=} \frac1N \sum_{i = 1}^N (y_i - (b + wx_i))^2
$$

에 대하여, $\theta_k = (b_k, w_k)$라 하고, 경사 하강법

$$
\theta_{k + 1} = \theta_k - \gamma \nabla L(\theta_k)
$$

을 이용하면, $\theta_k$는 ($\gamma$를 적당히 잡으면) 비용함수 $L$가 극솟값을 갖는 극소점으로 수렴합니다.
$\nabla L$을 직접 계산하면 다음과 같습니다.

$$
\begin{aligned}
    \frac{\partial L}{\partial b} &= \frac1N \sum_{i = 1}^N -2(y_i - (b + wx_i)) \\
    \frac{\partial L}{\partial w} &= \frac1N \sum_{i = 1}^N -2x_i(y_i - (b + wx_i)) \\
\end{aligned}
$$


In [ ]:
def loss_linear(params, x, y):
    # 뒤에서 설명할 자동 미분에서 나올 이유로
    # 파라미터값을 함수의 첫 번째 인자로 두는 것이 보통
    return ((y - (params["b"] + params["w"] * x))**2).mean()

def grad_loss_linear(params, x, y):
    # 직접 계산한 기울기
    diff = -2 * (y - (params["b"] + params["w"] * x))
    grads = {
        "b": diff.mean(),
        "w": (diff * x).mean(),
    }
    return grads

def update_params(params, grads, learning_rate):
    updated_params = {
        "b": params["b"] - learning_rate * grads["b"],
        "w": params["w"] - learning_rate * grads["w"],
    }
    return updated_params

# 이 함수에서 함수들을 부름
# 가장 바깥에 있는 함수만 JIT 컴파일해도 됨
@jax.jit
def step_manual_grad(params, x, y, learning_rate):
    grads = grad_loss_linear(params, x, y)
    params = update_params(params, grads, learning_rate)
    return params

def transpose_params_history(history):
    return {p: jnp.array([h[p] for h in history]) for p in params}

In [ ]:
n_epochs = 100
learning_rate = 0.3
params = {"b": jnp.array(3.0), "w": jnp.array(5.0)}

history = [params]
for i in range(1, n_epochs + 1):
    params = step_manual_grad(params, x_train, y_train, learning_rate)
    history.append(params)
history = transpose_params_history(history)

fig, ax = plt.subplots()
ax.plot(history["b"], ".-", label=R"$b$")
ax.plot(history["w"], ".-", label=R"$w$")
ax.set(xlabel="Epochs", ylabel="Parameter values", title="Training history")
ax.legend()

fig, ax = plt.subplots()
ax.scatter(x_train, y_train, c="C0", label="Data")
ax.axline([0, b_true], slope=w_true, c="C1", label="True line")
ax.axline(
    [0, params["b"]], slope=params["w"], c="C2", ls="--", label="Prediction"
)
ax.set(xlabel=R"$x$", ylabel=R"$y$", title="Result of training")
ax.legend()
pass

### 자동 미분 `jax.grad()`, `jax.jacfwd()`, `jax.jacrev()`

JAX는 일반적인 기계 학습 라이브러리의 역전파 방식과는 다른 인터페이스로, (위의 JIT과 같이) 함수를 함수로 변환하는 방식을 사용합니다.

```python
jac_f = jax.jacfwd(f)
# 또는
jac_f = jax.jacrev(f)
```

로 `f`의 야코비 행렬을 계산할 수 있습니다.

한 번 변환한 함수를 이어서 변환하는 것도 가능하므로, 미분을 두 번 이상 결합하여 고차원 미분을 하거나, JIT이나 (앞으로 나올) 벡터화 변환과도 결합 가능합니다.

JAX가 기울기(야코비 행렬)를 계산하는 방법은

- 독립 변수가 많고 함숫값의 차원이 작아서 야코비 행렬이 가로로 긴 경우에 적합한 reverse-mode `jax.jacrev()` (기계 학습은 보통 훈련 파라미터가 매우 많고 손실 함수의 함숫값은 스칼라이므로 reverse-mode가 적합. 보통 인공 신경망에서 말하는 역전파(backpropagation)와 동일)
- 독립 변수가 적고 함숫값의 차원이 커서 야코비 행렬이 세로로 긴 경우에 적합한 forward-mode (`jax.jacfwd()`)

가 있습니다.
`jax.grad()`는 특별히 종속 변수가 단 하나뿐(함숫값이 스칼라)인 함수에 reverse-mode를 적용하는 함수입니다.

```python
grad_f = jax.grad(f)
```

로 `f`의 기울기를 계산하는 새로운 함수를 만들 수 있고,

```python
value_and_grad_f = jax.value_and_grad(f)
```

로 `f`를 실행한 결과와, `f`의 기울기를 동시에 반환하는 함수를 만들 수도 있습니다.


In [ ]:
grad_sin = jax.grad(jnp.sin)

print(grad_sin(jnp.pi / 4))
print(jnp.cos(jnp.pi / 4))

In [ ]:
grad_grad_sin = jax.grad(jax.grad(jnp.sin))  # jax.grad(grad_sin)도 당연히 됨

print(grad_grad_sin(jnp.pi / 3))
print(-jnp.sin(jnp.pi / 3))

#### Pytree로 변수 묶어서 처리하기

`jax.grad`는 기본값으로 함수의 첫번째 인자에 대해서만 기울기를 구합니다.

In [ ]:
def f(b1, w1, b2, w2, x):
    x = b1 + w1 * x
    x = jnp.tanh(x)
    x = b2 + w2 * x
    return x

jax.grad(f)(0.1, 0.2, 0.3, 0.4, 0.5)  # => ∂f/∂b1만 반환

만약 다른 변수에 대해서도 기울기를 구하고 싶으면 `argnums` 인자를 이용해서 지정할 수는 있습니다.
하지만 이렇게 하는 경우 변수가 많아지면 복잡하고 코드의 유지 관리가 어려워진다는 문제가 있습니다.

In [ ]:
jax.grad(f, argnums=(0, 1, 2, 3))(0.1, 0.2, 0.3, 0.4, 0.5)

그래서 JAX에서는 이를 묶어서 처리할 수 있게 해 줍니다.

변수가 파이썬 리스트(`list`), 튜플(`tuple`), 딕셔너리(`dict`)의 임의의 조합으로 이루어져 있으면, 해당 변수 안에 있는 모든 값에 대해 기울기를 구한 후, 원래 변수의 구조와 똑같은 구조로 반환해 줍니다.

In [ ]:
def f(params, x):
    x = params["linear1"]["bias"] + params["linear1"]["weight"] * x
    x = jnp.tanh(x)
    x = params["linear2"]["bias"] + params["linear2"]["weight"] * x
    return x

params = {
    "linear1": {
        "bias": 0.1,
        "weight": 0.2,
    },
    "linear2": {
        "bias": 0.3,
        "weight": 0.4,
    },
}
jax.grad(f)(params, 0.5)

이러한 변수를 JAX에서는 PyTree라고 부르고, 이를 다룰 수 있는 많은 유틸리티 함수를 제공합니다.

예를 들어 `jax.tree.map()`은, PyTree 변수 안의 모든 단말 노드(leaf)에 일괄적으로 같은 함수를 적용해 줍니다.

In [ ]:
my_variable = {
    "linear_layer": {
        "b": jnp.zeros(10),
        "w": jnp.zeros((5, 10)),
    },
    "some_data": [
        jnp.zeros((3, 5, 7)),
        (
            jnp.zeros((10, 20)),
            jnp.zeros(50),
        ),
        jnp.zeros(8),
    ],
}

In [ ]:
jax.tree.map(lambda x: x.shape, my_variable)

In [ ]:
jax.tree.map(lambda x: x.size, my_variable)

#### 예제: 자동 미분을 이용한 경사 하강법

$\nabla L$을 직접 계산하는 대신 `jax.grad()`를 이용합니다.

In [ ]:
def loss_linear(params, x, y):
    return ((y - (params["b"] + params["w"] * x))**2).mean()

@jax.jit
def step_auto_grad(params, x, y, learning_rate):
    grads = jax.grad(loss_linear)(params, x, y)
    params = jax.tree.map(
        lambda param, grad: param - learning_rate * grad, params, grads
    )
    return params

In [ ]:
n_epochs = 100
learning_rate = 0.3
params = {"b": jnp.array(3.0), "w": jnp.array(5.0)}

history = [params]
for i in range(1, n_epochs + 1):
    params = step_auto_grad(params, x_train, y_train, learning_rate)
    history.append(params)
history = transpose_params_history(history)

fig, ax = plt.subplots()
ax.plot(history["b"], ".-", label=R"$b$")
ax.plot(history["w"], ".-", label=R"$w$")
ax.set(xlabel="Epochs", ylabel="Parameter values", title="Training history")
ax.legend()

fig, ax = plt.subplots()
ax.scatter(x_train, y_train, c="C0", label="Data")
ax.axline([0, b_true], slope=w_true, c="C1", label="True line")
ax.axline(
    [0, params["b"]], slope=params["w"], c="C2", ls="--", label="Prediction"
)
ax.set(xlabel=R"$x$", ylabel=R"$y$", title="Result of training")
ax.legend()
pass

### 자동 벡터화 `jax.vmap()`

`jax.vmap()`을 이용해 함수를 벡터화(병렬화)할 수 있습니다.
단순히 반복문을 순서대로 돌린 다음 합쳐 주는 것이 아니라, 정말로 병렬 처리가 되도록 잘 알아서 함수를 변환해 줍니다.

In [ ]:
def dot1d(x, y):
    # 벡터간 점곱: 함수가 1D 벡터만 처리할 수 있다고 가정
    if x.ndim != 1 or y.ndim != 1:
        raise ValueError("x and y should be 1D.")
    return jnp.dot(x, y)

In [ ]:
x = jnp.array([1.0, 2.0, 3.0, 4.0])
y = jnp.array([5.0, 6.0, 7.0, 8.0])

In [ ]:
dot1d(x, y)

In [ ]:
dot1d(x + 1, y + 1)

1D 벡터 `x`, `y` 대신, 2D 행렬 `xs`와 `ys`가 있을 때, 각각 행끼리 점곱을 일괄 계산하려고 합니다.
즉,

- `dot1d(xs[0, :], ys[0, :])`
- `dot1d(xs[1, :], ys[1, :])`
- `dot1d(xs[2, :], ys[2, :])`, ...
- ...

를 계산하고 싶습니다.
이 경우 위의 함수를 그대로 쓸 수는 없습니다.

In [ ]:
xs = jnp.stack((x, x + 1))
ys = jnp.stack((y, y + 1))

print(xs)
print(ys)

In [ ]:
# dot1d(xs, ys)  # Error

파이썬의 반복문으로 단순히 함수를 여러 번 불러서 합칠 수 있습니다.
보통은 이렇게 파이썬으로 반복하는 것은 성능이 떨어집니다.

In [ ]:
def loop_batched_dot1d(xs, ys):
    output = []
    for x, y in zip(xs, ys):
        output.append(dot1d(x, y))
    return jnp.stack(output)

loop_batched_dot1d(xs, ys)

직접 벡터화된 버전을 작성할 수 있습니다.
이 예의 경우에는 어렵지 않게 가능하지만, 복잡한 연산의 경우 단순하게 되지 않는 경우가 많습니다.

In [ ]:
def manually_vectorized_dot1d(xs, ys):
    return (xs * ys).sum(axis=1)

manually_vectorized_dot1d(xs, ys)

`jax.vmap()`을 이용해서 자동으로 벡터화할 수 있습니다.
기본값으로 맨 첫 번째 차원(`in_axes=0`)을 잘라서 계산해 주는데, `in_axes` 값을 지정해 주어서 방향을 바꿀 수도 있습니다.

In [ ]:
auto_vectorized_dot1d = jax.vmap(dot1d)

auto_vectorized_dot1d(xs, ys)
# => dot(xs[0, :], ys[0, :]), dot(xs[1, :], ys[1, :])

In [ ]:
auto_vectorized_columnwise_dot1d = jax.vmap(dot1d, in_axes=1)

auto_vectorized_columnwise_dot1d(xs, ys)
# => dot(xs[:, 0], ys[:, 0]), dot(xs[:, 1], ys[:, 1]), ...

입력 차원을 `None`으로 지정하면, 해당 인자는 병렬처리 하지 않고 모든 연산에 같은 값을 사용합니다.

In [ ]:
auto_x_only_vectorized_dot1d = jax.vmap(dot1d, in_axes=(0, None))

auto_x_only_vectorized_dot1d(xs, y)  # dot(xs[0, :], y), dot(xs[1, :], y)

조건만 만족하면 다른 함수 변환과 결합하여 사용할 수 있습니다.

In [ ]:
jitted_vectorized_dot1d = jax.jit(jax.vmap(dot1d))

jitted_vectorized_dot1d(xs, ys)

## 기타

### 재현 가능성과 random number generator

NumPy에서는 “random number generator”를 이용해 (유사)난수를 생성합니다.

NumPy에서 사용하는 난수 생성기 방식은, 내부에 현재 상태를 저장해 놓고 계속 변경해 가면서 사용합니다.

In [ ]:
rng = np.random.default_rng(78)  # 특정 seed로 rng의 시작 상태를 지정

print(rng.normal())
# => 난수를 하나 만들고, rng의 현재 상태를 바꿈

# 위와 똑같은 코드인데도 rng의 상태가 다르므로 값이 바뀜
print(rng.normal())

위와 같은 방식은 순수 함수와 변경 불가능성의 장점을 이용하는 JAX의 전략과는 잘 맞지 않으며, 병렬 연산을 할 때 같은 결과를 재현하기도 어렵습니다.

그래서 JAX에서는 대신 subkey splitting 방식을 사용합니다.

- 난수를 생성할 때는 항상 (단순한 데이터일 뿐인) “key”를 넘겨 주어야 합니다.
  - 이 키는 자동으로 바뀌거나 하지 않습니다.
- 난수를 여러 번 생성하고 싶으면, 키를 미리 “쪼개서” 사용해야 합니다.

In [ ]:
key = jax.random.key(78)

# jax.random.normal도 일반적인 다른 JAX 함수와 마찬가지로 순수 함수.
# Key가 동일하면 항상 결과가 동일함.
# jax.random.normal이 자동으로 값이 바뀌거나, key 값이 자동으로 바뀌지 않음
print(key)
print(jax.random.normal(key))

print(key)
print(jax.random.normal(key))

In [ ]:
key = jax.random.key(78)

key1, key2 = jax.random.split(key)  # key 씀. 다시 쓰면 안됨
# => key1, key2가 새로 생김
print(jax.random.normal(key2))  # key2 씀. key1은 아직 안 씀

key3, key4, key5 = jax.random.split(key1, 3)  # key1을 씀
# key3, key4, key5가 새로 생김
print(jax.random.normal(key4))  # key4 씀
print(jax.random.normal(key5))  # key5 씀

# key3 아직 쓴 적이 없으므로 써도 됨...

In [ ]:
# 위 셀과 동일하지만, key 변수 이름을 덮어쓰기해서 사용함.
# 같은 key 값을 여러 번 쓰는 것이 아님.
key = jax.random.key(78)

key, subkey = jax.random.split(key)
print(jax.random.normal(subkey))

key, subkey1, subkey2 = jax.random.split(key, 3)
print(jax.random.normal(subkey1))
print(jax.random.normal(subkey2))

# key 계속 사용...